In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.data.dataloader import default_collate
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import torch
import pickle


# Append the project dir to path
sys.path.append(os.path.join("..", "..", ".."))
from data_pipeline.utils import train_test_split, create_metadata_df, get_sample_weights_of_dataset, measurements_to_df
from data_pipeline.dataset_xy import CARLADatasetXY
from data_pipeline.dataset_xy_opt import CARLADatasetXYOpt
from data_pipeline.data_sampler import BranchPerCommandSampler
from data_pipeline.data_preprocessing import preprocessing
from models.resnet_baseline.architectures_v3 import Resnet_Baseline_V3, Resnet_Baseline_V3_Dropout
from models.resnet_baseline.architectures_v5 import Resnet_Baseline_V5
from models.resnet_lidar.lidar_v1 import Resnet_Lidar_V1, Resnet_Lidar_V1_Dropout, Resnet_Lidar_V1_Dropout_2
from models.model_trainer import ModelTrainer

## Choose training settings

In [ ]:
# Set data balancing options (if both false, then no balancing is applied)
use_balance_by_loss_weighting = False
use_balance_by_over_under_sampling = False

assert not use_balance_by_loss_weighting or not use_balance_by_over_under_sampling

In [ ]:
# Train additionally on the noisy data
use_data_noisy = True

path_data_noisy = None
if use_data_noisy:
    path_data_noisy = os.path.join("..", "..", "..", "data", "Noise-Dataset")

## Create Datasets

In [ ]:
path_data = os.path.join("..", "..", "..", "data", "data")

config_xy = {"used_inputs": ["rgb", "lidar_bev", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# config_xy = {"used_inputs": ["rgb", "measurements"], 
#         "used_measurements": ["speed", "waypoints", "command"],
#         "y": ["waypoints"],
#         "seq_len": 1
#         }

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.has_mps else 'cpu')
batch_size = 64

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])
df_meta_data_noisy = None
if use_data_noisy:
    df_meta_data_noisy = create_metadata_df(path_data_noisy, config_xy["used_inputs"])

# Train/test split
train_test_config = {
    "train": ['Town00', 'Town01', 'Town02', 'Town03', 'Town04', 'Town05', 'Town07', 'Town08', 'Town09', 'Town10'],
    "test": ['Town06']
}
df_meta_data_train, df_meta_data_test_1, df_meta_data_test_2 = train_test_split(df_meta_data, towns_intersect=train_test_config, df_meta_data_noisy=df_meta_data_noisy)

# Decrease train/test size for quick test run
# df_meta_data_train = df_meta_data_train.head(5 * batch_size)
# df_meta_data_test_1 = df_meta_data_test_1.head(5 * batch_size)
# df_meta_data_test_2 = df_meta_data_test_2.head(5 * batch_size)

# Create Dataset & DataLoader
dataset_train = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_train, config=config_xy)
dataset_test_1 = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_test_1, config=config_xy)
dataset_test_2 = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_test_2, config=config_xy)

# dataset_train = CARLADatasetXYOpt(df_meta_data_train)
# dataset_test_1 = CARLADatasetXYOpt(df_meta_data_test_1)
# dataset_test_2 = CARLADatasetXYOpt(df_meta_data_test_2)

## Generate sample weights to be passed to ModelTrainer

In [ ]:
def save_sample_weights(sample_weights):
    with open('sample_weights.pickle', 'wb') as handle:
        pickle.dump(sample_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_sample_weights():
    with open('sample_weights.pickle', 'rb') as handle:
        b = pickle.load(handle)
    return b

In [ ]:
sample_weights = None
if use_balance_by_loss_weighting or use_balance_by_over_under_sampling:
    # Dictionary that saves all weights to all y variables 
    sample_weights = get_sample_weights_of_dataset(dataset_train, num_bins=10, multilabel_option=use_balance_by_over_under_sampling) # TODO: Hacky False to try prob balacing only on steer
    # sample_weights = load_sample_weights()
    print(sample_weights.keys())

In [ ]:
# TODO: Hacky False to try prob balacing only on steer
# sample_weights = {"multilabel": sample_weights["steer"]}

In [ ]:
# TODO: Hacky cmd based sample weights
#cmd_counts_serd = df_meas_train["command"].value_counts().sort_index()
#sample_weights = {"multilabel": np.array([1 / cmd_counts_ser.iloc[item-1] for item in df_meas_train["command"].values])}

## Create DataLoaders

In [ ]:
# df_meas_train = measurements_to_df(df_meta_data_train)
# df_meas_train["probs"] = sample_weights["multilabel"]

In [ ]:
weighted_random_sampler = None
shuffle = True
if use_balance_by_over_under_sampling:
    weighted_random_sampler = WeightedRandomSampler(weights=sample_weights["multilabel"], num_samples=dataset_train.__len__(), replacement=True)
    shuffle = False

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, num_workers=0, shuffle=shuffle, sampler=weighted_random_sampler)
dataloader_test_1 = DataLoader(dataset_test_1, batch_size=batch_size, num_workers=0, shuffle=False, )
dataloader_test_2 = DataLoader(dataset_test_2, batch_size=batch_size, num_workers=0, shuffle=False, )

# Attempt to directly initialize tensors on device in the DataLoader
# collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x))
# collate_fn=lambda x: list(map(lambda x: x.to(device), default_collate(x))

In [ ]:
len(dataloader_train)

In [ ]:
len(dataloader_test_1)

In [ ]:
for x, y, idx in dataloader_test_1:
    break

# Create ModelTrainer & run it

In [ ]:
# model = Resnet_Lidar_V1_Dropout(0.25)
# model = Resnet_Baseline_V3_Dropout(0.25)
# model = Resnet_Baseline_V3()
model = Resnet_Lidar_V1_Dropout_2()
if not use_balance_by_loss_weighting:
    sample_weights = None

In [ ]:
#sum_object = summary(model, [(3, 160, 960), (7, ), (1,)], 64) # (3, 88, 244)

In [ ]:
# Must be ordered alphabetically (i.e. the same like sample_weights keys)
loss_fns_dict = {"brake": nn.L1Loss(reduction='none'), "steer": nn.L1Loss(reduction='none'), "throttle": nn.L1Loss(reduction='none')}
loss_fn_weights = {"brake": 0.05, "steer": 0.45, "throttle": 0.5}
# loss_fns_dict = {"waypoints": nn.L1Loss(reduction='none')}
# loss_fn_weights = {"waypoints": 1}

model_trainer = ModelTrainer(
    model=model,
    optimizer=optim.Adam(model.parameters(), lr=0.0001),
    loss_fns=loss_fns_dict,
    loss_fn_weights=loss_fn_weights,
    n_epochs=10,
    dataloader_train=dataloader_train,
    dataloader_test=dataloader_test_2,
    sample_weights=sample_weights,
    preprocessing=preprocessing,
    upload_tensorboard=True
    )

In [ ]:
model_trainer.run()

In [ ]:
model_trainer.df_performance_stats[["val_brake_loss",	"val_steer_loss",	"val_throttle_loss"]].mean(axis=1)

In [ ]:
model_trainer.df_performance_stats

In [ ]:
model_trainer.df_performance_stats.to_csv("test.csv", index=False)

# Investigating model predictions (errors)
To be moved in an extra module at some time ...

In [ ]:
model = Baseline_V3()
model.load_state_dict(torch.load("baseline_v3_7_hours.pt"))

In [ ]:
model = model_trainer.model.to(torch.device("cpu"))
torch.save(model.state_dict(), "resnet_baseline_v3_4_10_epochs_loss_balanced_noisy_2.pt")

In [ ]:
model_trainer = ModelTrainer(
    model=model,
    optimizer=optim.Adam(model.parameters(), lr=0.0001),
    loss_fn=nn.L1Loss(),
    n_epochs=10,
    dataloader_train=dataloader_train,
    dataloader_test=dataloader_test,
    preprocessing=preprocessing,
    upload_tensorboard=True
    )

In [ ]:
y_true_list, y_pred_list = model_trainer.get_dataset_predictions()

In [ ]:
df_true = pd.DataFrame(np.transpose(y_true_list), columns=dataset_test.y)
df_pred = pd.DataFrame(np.transpose(y_pred_list), columns=dataset_test.y)

In [ ]:
df_pred.hist(bins=20)

In [ ]:
df_residuals = df_true - df_pred

In [ ]:
df_residuals.hist(bins=20)

## Experiment which operators/function can be executed on device

In [ ]:
sum([torch.tensor([1, 2], device=torch.device("mps")), torch.tensor([1, 2], device=torch.device("mps"))])

In [ ]:
# Preprocessing could be done on GPU but not on MPS (Apple)
preprocessing["rgb"](torch.rand((3, 160, 960), device=torch.device("mps")))

In [ ]:
torch.tensor([1, 2], device=torch.device("mps")).device

In [ ]:
test_dict = {"t1": torch.tensor([1, 2, 3])}
# [test_dict[key].to(torch.device("mps")) for key in test_dict]
for key in test_dict:
    test_dict[key] = test_dict[key].to(torch.device("mps"))

In [ ]:
torch_values = torch.tensor([0, 1, 2, 3, 4], device=torch.device("mps"))
torch_IDX = torch.tensor([0, 2])

In [ ]:
torch_values[torch_IDX]

In [ ]:
dirs = os.listdir("runs")
dirs_creation_time = [os.path.getctime(os.path.join("runs", dir)) for dir in dirs]


In [ ]:
[el[0] for el in sorted(zip(dirs, dirs_creation_time), key=lambda x: x[1])][-1]

In [ ]:
os.path.getctime('runs/Feb03_15-22-49_MBPvonJulian2.fritz.box')